In [ ]:
import os
import json
import matplotlib
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import StratifiedGroupKFold, GroupKFold

In [ ]:
DATA_PATH = "../raw_data/"

In [ ]:
topics = pd.read_csv(DATA_PATH + "topics.csv")
content = pd.read_csv(DATA_PATH + "content.csv")
correlations = pd.read_csv(DATA_PATH + "correlations.csv")

topics = topics[topics.has_content==True]

## Split

In [ ]:
topics_train = topics[topics['category'] == "source"][["id"]]
topics_train["fold"] = "train"

In [ ]:
topics_val = topics[topics['category'] != "source"].reset_index(drop=True)

sgkf = StratifiedGroupKFold(random_state=1773,
                            n_splits=4,
                            shuffle=True)
split_idxs = list(sgkf.split(topics_val["id"],
                             topics_val["language"],
                             groups=topics_val["id"]))[0]

split_idxs

In [ ]:
topics_add_train = topics_val.iloc[split_idxs[0]].reset_index(drop=True)[["id"]]
topics_add_train["fold"] = "train"

In [ ]:
topics_train = pd.concat([topics_train, topics_add_train], ignore_index=True)

In [ ]:
topics_holdout = topics_val.iloc[split_idxs[1]].reset_index(drop=True)[["id"]]
topics_holdout["fold"] = "test"

In [ ]:
topics[topics.id.isin(topics_train.id)].language.value_counts()

In [ ]:
topics[topics.id.isin(topics_add_train.id)].language.value_counts()

In [ ]:
topics[topics.id.isin(topics_holdout.id)].language.value_counts()

In [ ]:
split_df = pd.concat([topics_train, topics_holdout], ignore_index=True)

In [ ]:
split_df.to_csv('train_test_splits.csv', index=False)

Done !